In [1]:
class color():
    
    RED = '\033[31m'
    GREEN = '\033[32m'
    YELLOW = '\033[33m'
    BLUE = '\033[34m'
    MAGENTA = '\033[35m'
    CYAN = '\033[36m'
    RED_WHITE = '\033[41m'
    GREEN_RED = '\033[31;42m'
    YELLOW_RED = '\033[33;43m'
    BLUE_WHITE = '\033[44m'
    MAGENTA_BLUE = '\033[35;45m'
    CYAN_RED = '\033[31;46m'
    END = '\033[0m'

In [3]:
#圓餅圖
import matplotlib
import matplotlib.pyplot as plt 
myfont = matplotlib.font_manager.FontProperties(fname='c:\\windows\\fonts\\msyh.ttc', size=14)

def pie_Chart(label,size):
    
    plt.ion()
    
    labels = label
    sizes = []
    explode = []
    
    sum_size = 0
    for i in size:
        sum_size += i
        explode.append(0)
    for i in size:
        sizes.append(round((i/sum_size)*100,2))
    
    if len(sizes) == 0:
        return
        
    explode[0] = 0.05
    explode = tuple(explode)
    #print (labels)
    #print (sizes)
    for i in range(len(labels)):
        print (labels[i]+':'+str(sizes[i])+'%',end = ' ')
    

    #调节图形大小，宽，高
    plt.figure(figsize=(6,9))
    #定义饼状图的标签，标签是列表
    #labels = ['第一部分','第二部分','第三部分']
    #每个标签占多大，会自动去算百分比
    #sizes = [30,40,30]
    #colors = ['red','yellowgreen','lightskyblue']
    colors = ['b','yellowgreen','r','c','m','y','w']
    #将某部分爆炸出来， 使用括号，将第一块分割出来，数值的大小是分割出来的与其他两块的间隙
    #explode = (0,0.05,0)

    patches,l_text,p_text = plt.pie(sizes,explode=explode,labels=labels,colors=colors,
                                    labeldistance = 1.1,autopct = '%3.1f%%',shadow = False,
                                    startangle = 90,pctdistance = 0.6)

    #labeldistance，文本的位置离远点有多远，1.1指1.1倍半径的位置
    #autopct，圆里面的文本格式，%3.1f%%表示小数有三位，整数有一位的浮点数
    #shadow，饼是否有阴影
    #startangle，起始角度，0，表示从0开始逆时针转，为第一块。一般选择从90度开始比较好看
    #pctdistance，百分比的text离圆心的距离
    #patches, l_texts, p_texts，为了得到饼图的返回值，p_texts饼图内部文本的，l_texts饼图外label的文本

    #改变文本的大小
    #方法是把每一个text遍历。调用set_size方法设置它的属性
    for t in l_text:
        t.set_size=(30)
        t.set_fontproperties(myfont)
    for t in p_text:
        t.set_size=(20)
        #t.set_fontproperties(myfont)
    # 设置x，y轴刻度一致，这样饼图才能是圆的
    plt.axis('equal')
    plt.legend(prop=myfont,loc=1)
    plt.show()
    plt.pause(30)
    plt.close()


In [2]:
import pymysql
import codecs
import os
import time

modal = [] #情態類別 
modal_color = {} #情態類別顏色
word_modal = {} #情態詞:類別
exception = {} #情態詞:複數類別

def color_set(model):
    modal_color[modal[0]] = color.RED
    modal_color[modal[1]] = color.GREEN
    modal_color[modal[2]] = color.YELLOW
    modal_color[modal[3]] = color.BLUE
    modal_color[modal[4]] = color.MAGENTA
    modal_color[modal[5]] = color.CYAN
    modal_color[modal[6]] = color.RED_WHITE
    modal_color[modal[7]] = color.GREEN_RED
    modal_color[modal[8]] = color.YELLOW_RED
    modal_color[modal[9]] = color.BLUE_WHITE
    modal_color[modal[10]] = color.MAGENTA_BLUE
    modal_color[modal[11]] = color.CYAN_RED
    
    for i in modal:
        print (modal_color[i]+i+color.END)
    
    
def sql_search(sql_command):

    connection = pymysql.connect(host="127.0.0.1", user="redwing", passwd="", db="word",charset='utf8')

    result = []

    try:
        with connection.cursor() as cursor:
            cursor.execute(sql_command)

            for i in cursor:
                result.append(i) 

            return result
    finally:
        connection.close()

def make_modal():

    for i in sql_search("SELECT distinct `class` FROM `word`"):
        modal.append(i[0])
        
    color_set(modal)

    for i in sql_search("select * from `word`"):
        if i[2] in exception: #情態詞在例外內
            exception[i[2]].append(i[1])
        elif i[2] not in word_modal: #情態詞不在索引內
            word_modal[i[2]] = i[1]
        elif i[1] != word_modal[i[2]]: #情態詞在索引內重複且不相等
            exception[i[2]] = [word_modal[i[2]],i[1]]
            word_modal.pop(i[2])
            
def get_article(file_address):
    
    path = 'C:\\Users\\user\\Desktop\\article\\'
    result = {}
    
    with codecs.open(path+file_address,'rb','utf8') as f:
        content = f.readlines()
        
        for line in content:
            if line[0] == '#':
                print (line.strip())
                continue
            try:
                line = line.split()
                
                for word in line:
                    word = word.split('(')[0]
                    if word in exception:
                        print ('['+word+'(例外)]',end='')
                        if '例外'  not in result:
                            result['例外'] = [word]
                        else:
                            result['例外'].append(word)
                    elif word in word_modal:
                        print (modal_color[word_modal[word]]+'['+word+']'+color.END,end='')
                        if word_modal[word] not in result:
                            result[word_modal[word]] = [word]
                        else:
                            result[word_modal[word]].append(word)
                    else:
                        print (word,end='')
            except:
                print (line)
    return result

make_modal()

認知-猜測 epistemic-conjecture
認知-斷定 epistemic-assertive
認知-真偽 epistemic-alethic
認知-感知 epistemic-sensory
認知-引述 epistemic-reportive
義務-允許 denotic-permissive
義務-要求 denotic-obligative
義務-承諾 denotic-promise
動力-潛力 dynamic-capacity 
動力-意願 dynamic-volition
評價-預料 evaluative-presuppositio
評價-心願 evaluative-wish


In [8]:
find = 0
find = input()
while(find != 'x'):
    result = get_article(str(find).zfill(5)+'.txt')
    
    pie_label = []
    pie_size = []
    
    print ()
    for i in result:
        if i != '例外':
            print (i,len(result[i]),end=' ')
            pie_label.append(i)
            pie_size.append(len(result[i]))
            temp = []
            
            print ('[',end = ' ')
            for j in result[i]:
                if j not in temp:
                    temp.append(j)
                    print (j+':'+str(result[i].count(j)),end = ' ')
            print (']')
    if '例外' in result:
        print ('例外',len(result['例外']),end = ' ')
        pie_label.append('例外')
        pie_size.append(len(result['例外']))
        temp = []
        
        print ('[',end = ' ')
        for j in result['例外']:
            if j not in temp:
                temp.append(j)
                print (j+':'+str(result['例外'].count(j)),end = ' ')
        print (']')
        
    pie_Chart(pie_label,pie_size)
    
    find = input()

7423
北台灣焦點‧兒女[要(例外)]養也[要(例外)]教‧宜蘭：為[免]青少年誤蹈法網，宜蘭縣警察局昨日辦理警光營活動，加強學生們對法律常識和安毒的認識，以免誤入歧途，縣刑警隊副隊長馮富松有感而發指出，青少年犯罪與家長有密切關係，因此為人父母[應]負起教養之責，養而不教，父之過！古有明訓。
義務-允許 denotic-permissive 1 [ 免:1 ]
義務-要求 denotic-obligative 1 [ 應:1 ]
例外 2 [ 要:2 ]
義務-允許 denotic-permissive:25.0% 義務-要求 denotic-obligative:25.0% 例外:50.0% 

C:\Users\user\Anaconda2\envs\py34\lib\site-packages\matplotlib\backend_bases.py:2437: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


12654
#鼓勵女性從事研究 學者籲完善幼托系統
「傑出學者給年輕學子的６７封信」中的６７位作者，清一色全[是]男性，而所有９８位傑出人才獎得主也只有２位女性，顯示我國研究領域男女比例差異懸殊。中研院長、傑出人才發展基金會董事長李遠哲以法國為例，呼籲我國建置完善幼托系統，[讓]女性[能(例外)]放心從事研究工作。傑出人才基金會執行長蕭新煌也[希望]出版界[能(例外)]為優秀女性研究人員出版傳記，樹立典範。蕭新煌指出，歷屆傑出人才獎約有５００至６００人提出申請，其中女性比例極低，這[是]傑出人才基金會未來[要(例外)]努力的目標；教育部長黃榮村則表示，目前大專院校的女性教師大約佔四分之一至三分之一左右，比例上[似乎]沒有科學界懸殊。李遠哲以日前與法國學術團體交流的經驗指出，法國女性研究人員比例較高，除了大學男女分校外，法國具有女性研究人員的典範，具有鼓勵女性學生從事研究工作的作用；另外，法國具有完善的幼托系統，[可以(例外)][讓]女性研究員無後顧之憂，[可以(例外)]全心投入研究工作，國內[可以(例外)]朝樹立典範、建置完善幼托系統努力。蕭新煌則表示，在樹立典範方面，除國家級獎項[可(例外)]加強審查工作外，出版社也[可(例外)]朝為優秀女研究員出版傳記的方向努力。蕭新煌表示，未來不排除出版「女傑出學者給年輕學子的信」，鼓勵有志從事研究工作的女性學生。
義務-允許 denotic-permissive 2 [ 讓:2 ]
認知-感知 epistemic-sensory 1 [ 似乎:1 ]
認知-真偽 epistemic-alethic 2 [ 是:2 ]
動力-意願 dynamic-volition 1 [ 希望:1 ]
例外 8 [ 能:2 要:1 可以:3 可:2 ]
義務-允許 denotic-permissive:14.29% 認知-感知 epistemic-sensory:7.14% 認知-真偽 epistemic-alethic:14.29% 動力-意願 dynamic-volition:7.14% 例外:57.14% 5698
#新聞報導及校園事件
八十五年全國文藝季正在全省各縣市陸續展開。雲林縣今年[將]安排「跳躍花鼓，大廓情」的鄉土藝文活動，邀您五月二十五到三十一日來同享鄉土古早味。雲林縣立文化中心主任蔡春風表示，今年文藝季雲林縣安排在褒忠鄉大